# 实验 3.3：对抗样本迁移攻击（黑盒攻击）

## 🎯 学习目标

完成本实验后，你将能够：
- ✅ 理解对抗样本的迁移性（Transferability）原理
- ✅ 实现黑盒攻击场景下的迁移攻击策略
- ✅ 分析不同模型架构间的迁移成功率
- ✅ 探索提高迁移性的高级技巧（多模型集成攻击）

## 📚 前置知识

- 完成实验 3.1（FGSM）和实验 3.2（PGD）
- 理解白盒攻击与黑盒攻击的区别
- 了解不同神经网络架构（ResNet, VGG, DenseNet）

## 🖥️ 实验环境

| 项目 | 说明 |
|------|------|
| 平台 | 腾讯 Cloud Studio |
| GPU | NVIDIA Tesla T4（16GB）|
| 模型 | ResNet-18/34, VGG-16, DenseNet-121 |
| 框架 | PyTorch 2.0+ |

## ⏱️ 预计时间：35 分钟

## 📝 填空说明

本实验共 **5 个填空练习**，难度：⭐⭐⭐☆☆

## 核心概念回顾

### 白盒 vs 黑盒攻击

| 攻击类型 | 知道什么 | 不知道什么 | 难度 |
|---------|---------|-----------|------|
| **白盒攻击** | 模型结构、参数、梯度 | - | 容易 |
| **黑盒攻击** | 只能查询输入输出 | 模型内部细节 | 困难 |

### 迁移攻击原理

**核心思想**：在已知的替代模型（Surrogate Model）上生成对抗样本，利用对抗样本的迁移性攻击未知的目标模型（Target Model）。

**攻击流程**：
```
1. 训练/获取替代模型（与目标模型类似）
2. 在替代模型上用白盒方法（FGSM/PGD）生成对抗样本
3. 将对抗样本送入目标模型
4. 观察是否成功欺骗目标模型
```

**为什么对抗样本具有迁移性？**
- 不同模型学到的决策边界存在相似性
- 高维空间中的脆弱方向具有共性
- 模型都在相似的数据分布上训练

**影响迁移性的因素**：
- 模型架构相似度（ResNet → ResNet 迁移率高）
- 扰动强度（ε 越大，迁移性越强）
- 攻击方法（PGD > FGSM）
- 训练数据的相似性

## 第一部分：环境准备与多模型加载

In [ ]:
# 导入必要的库
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from torchvision import models, transforms

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 设置随机种子
torch.manual_seed(42)
np.random.seed(42)

# ImageNet 标准化参数
normalize = transforms.Normalize(
    mean=[0.485, 0.456, 0.406],
    std=[0.229, 0.224, 0.225]
)

print("=" * 50)
print("🔧 环境准备完成")
print("=" * 50)

In [ ]:
# 加载多个预训练模型
# 策略：使用一个模型作为替代模型生成对抗样本，测试在其他模型上的迁移效果

print("=" * 50)
print("📦 加载多个预训练模型")
print("=" * 50)
print("正在下载模型权重（首次运行可能需要几分钟）...\n")

# 替代模型：用于生成对抗样本（假设攻击者可以访问）
print("1️⃣ 加载替代模型...")
surrogate_model = models.resnet18(pretrained=True)
surrogate_model.eval()
print("   ✓ 替代模型: ResNet-18 (~11M 参数)")

# 目标模型：攻击者想要欺骗的模型（黑盒，攻击者不知道内部结构）
print("\n2️⃣ 加载目标模型...")
target_models = {
    'ResNet-34': models.resnet34(pretrained=True),
    'VGG-16': models.vgg16(pretrained=True),
    'DenseNet-121': models.densenet121(pretrained=True),
}

for name, model in target_models.items():
    model.eval()
    param_count = sum(p.numel() for p in model.parameters()) / 1e6
    print(f"   ✓ 目标模型: {name} (~{param_count:.1f}M 参数)")

print("\n" + "=" * 50)
print("✓ 所有模型加载完成！")
print(f"  替代模型: 1 个")
print(f"  目标模型: {len(target_models)} 个")
print("=" * 50)

In [ ]:
# 创建测试图像和辅助函数
def create_test_image():
    """创建合成测试图像"""
    np.random.seed(42)
    img = np.random.rand(224, 224, 3) * 0.3 + 0.35
    center_x, center_y = 112, 112
    for i in range(224):
        for j in range(224):
            dist = np.sqrt((i - center_x)**2 + (j - center_y)**2)
            if dist < 60:
                img[i, j] = [0.1, 0.1, 0.1]
            elif dist < 80:
                img[i, j] = [0.9, 0.9, 0.9]
    return torch.tensor(img, dtype=torch.float32).permute(2, 0, 1)

def predict(model, img_tensor):
    """
    使用模型进行预测
    
    参数:
        model: 预训练模型
        img_tensor: 图像张量 [C, H, W]
    
    返回:
        pred_class: 预测类别
        confidence: 预测置信度
    """
    input_tensor = normalize(img_tensor).unsqueeze(0)
    with torch.no_grad():
        output = model(input_tensor)
    probs = torch.softmax(output, dim=1)
    pred_class = output.argmax(dim=1).item()
    confidence = probs[0, pred_class].item()
    return pred_class, confidence

# 创建测试图像
original_image = create_test_image()

# 在所有模型上测试原始图像
print("=" * 65)
print("📊 原始图像在各模型上的预测结果")
print("=" * 65)
print(f"{'模型名称':<20} {'预测类别':<15} {'置信度':<15}")
print("-" * 65)

# 替代模型预测
surrogate_class, surrogate_confidence = predict(surrogate_model, original_image)
print(f"{'ResNet-18 (替代)':<20} {surrogate_class:<15} {surrogate_confidence:<15.2%}")

# 目标模型预测
for name, model in target_models.items():
    pred, conf = predict(model, original_image)
    print(f"{name + ' (目标)':<20} {pred:<15} {conf:<15.2%}")

print("=" * 65)

## 第二部分：在替代模型上生成对抗样本

迁移攻击的第一步：使用白盒方法在替代模型上生成对抗样本。

In [ ]:
# 实现 PGD 攻击函数（用于生成对抗样本）
def pgd_attack(model, image, label, epsilon, alpha, num_steps):
    """
    在指定模型上执行 PGD 攻击
    
    参数:
        model: 目标模型
        image: 原始图像 [C, H, W]
        label: 原始标签
        epsilon: 最大扰动范围
        alpha: 每步步长
        num_steps: 迭代次数
    
    返回:
        adversarial_image: 对抗样本
    """
    adv_image = image.clone().detach()
    
    for step in range(num_steps):
        adv_image.requires_grad = True
        normalized = normalize(adv_image).unsqueeze(0)
        output = model(normalized)
        loss = nn.CrossEntropyLoss()(output, torch.tensor([label]))
        
        model.zero_grad()
        loss.backward()
        
        gradient = adv_image.grad.data
        adv_image = adv_image.detach() + alpha * torch.sign(gradient)
        
        # 投影到 epsilon-ball
        perturbation = adv_image - image
        perturbation = torch.clamp(perturbation, -epsilon, epsilon)
        adv_image = torch.clamp(image + perturbation, 0, 1)
    
    return adv_image.detach()

print("✓ PGD 攻击函数定义完成！")

In [ ]:
# 设置攻击参数
epsilon = 0.03      # 最大扰动范围
alpha = 0.01        # 每步步长
num_steps = 20      # 迭代次数

# ========== 填空 1：在替代模型上生成对抗样本 ==========
# 
# 🎯 任务：使用 PGD 攻击在替代模型上生成对抗样本
# 
# 💡 提示：
#   - 替代模型是 surrogate_model
#   - 原始图像是 original_image
#   - 原始标签是 surrogate_class
#   - 使用上面定义的参数 epsilon, alpha, num_steps
# 
# 难度：⭐⭐☆☆☆

adversarial_image = ___________  # 期望：pgd_attack(surrogate_model, original_image, surrogate_class, epsilon, alpha, num_steps)

# 验证对抗样本在替代模型上的效果
adv_class, adv_confidence = predict(surrogate_model, adversarial_image)

print("=" * 50)
print("⚔️  替代模型上的攻击结果")
print("=" * 50)
print(f"  原始预测: 类别 {surrogate_class}, 置信度 {surrogate_confidence:.2%}")
print(f"  攻击后预测: 类别 {adv_class}, 置信度 {adv_confidence:.2%}")
print(f"  扰动强度: ε = {epsilon}")

if adv_class != surrogate_class:
    print(f"  攻击结果: ✓ 成功！预测从类别 {surrogate_class} 改变为类别 {adv_class}")
else:
    print(f"  攻击结果: ✗ 失败，预测未改变")
print("=" * 50)

## 第三部分：测试对抗样本的迁移性

关键问题：在 ResNet-18 上生成的对抗样本，能否欺骗其他架构的模型？

In [ ]:
# 测试对抗样本在各目标模型上的迁移效果
print("=" * 75)
print("🎯 迁移攻击测试结果")
print("=" * 75)
print(f"{'目标模型':<18} {'原始预测':<15} {'攻击后预测':<15} {'迁移结果':<15}")
print("-" * 75)

transfer_results = {}

for name, model in target_models.items():
    # 获取原始图像的预测
    orig_pred, orig_conf = predict(model, original_image)
    
    # ========== 填空 2：获取对抗样本在目标模型上的预测 ==========
    # 
    # 🎯 任务：将对抗样本送入目标模型，观察迁移效果
    # 
    # 💡 提示：
    #   - 使用 predict 函数
    #   - 参数是当前的 model 和 adversarial_image
    #   - 返回预测类别和置信度
    # 
    # 难度：⭐☆☆☆☆
    
    adv_pred, adv_conf = ___________  # 期望：predict(model, adversarial_image)
    
    # 判断迁移是否成功（预测类别改变即为成功）
    success = orig_pred != adv_pred
    transfer_results[name] = {
        'success': success,
        'orig_pred': orig_pred,
        'adv_pred': adv_pred,
        'orig_conf': orig_conf,
        'adv_conf': adv_conf
    }
    
    status = "✓ 成功" if success else "✗ 失败"
    status_color = '\033[92m' if success else '\033[91m'
    reset_color = '\033[0m'
    
    print(f"{name:<18} {f'类别 {orig_pred}':<15} {f'类别 {adv_pred}':<15} {status:<15}")

# 计算总体迁移成功率
success_count = sum(1 for r in transfer_results.values() if r['success'])
total_count = len(transfer_results)
transfer_rate = success_count / total_count * 100

print("-" * 75)
print(f"📊 迁移成功率: {success_count}/{total_count} ({transfer_rate:.1f}%)")
print("=" * 75)

In [ ]:
# 可视化迁移攻击效果
fig = plt.figure(figsize=(16, 10))

# 第一行：原始图像、对抗样本和扰动
ax1 = plt.subplot(2, 4, 1)
ax1.imshow(original_image.permute(1, 2, 0).numpy())
ax1.set_title(f"原始图像\n类别: {surrogate_class}", fontsize=11, color='green')
ax1.axis('off')

ax2 = plt.subplot(2, 4, 2)
ax2.imshow(adversarial_image.permute(1, 2, 0).numpy())
ax2.set_title(f"对抗样本\n在 ResNet-18 上生成", fontsize=11, color='orange')
ax2.axis('off')

ax3 = plt.subplot(2, 4, 3)
perturbation = adversarial_image - original_image
perturbation_vis = (perturbation - perturbation.min()) / (perturbation.max() - perturbation.min() + 1e-8)
ax3.imshow(perturbation_vis.permute(1, 2, 0).numpy())
ax3.set_title(f"对抗扰动（放大）\nε = {epsilon}", fontsize=11)
ax3.axis('off')

# 迁移成功率统计
ax4 = plt.subplot(2, 4, 4)
success_counts = [1 if r['success'] else 0 for r in transfer_results.values()]
model_names = list(transfer_results.keys())
colors = ['green' if s else 'red' for s in success_counts]
ax4.bar(range(len(model_names)), success_counts, color=colors, alpha=0.7)
ax4.set_xticks(range(len(model_names)))
ax4.set_xticklabels([name.split('-')[0] for name in model_names], rotation=45)
ax4.set_ylim(0, 1.2)
ax4.set_ylabel('迁移成功 (1) / 失败 (0)')
ax4.set_title(f'迁移成功率: {transfer_rate:.0f}%', fontsize=11, fontweight='bold')
ax4.grid(axis='y', alpha=0.3)

# 第二行：各目标模型的详细结果
for idx, (name, result) in enumerate(transfer_results.items()):
    ax = plt.subplot(2, 4, 5 + idx)
    ax.imshow(adversarial_image.permute(1, 2, 0).numpy())
    
    color = 'red' if result['success'] else 'blue'
    status = "迁移成功 ✓" if result['success'] else "迁移失败 ✗"
    
    title = f"{name}\n{result['orig_pred']} → {result['adv_pred']}\n{status}"
    ax.set_title(title, fontsize=10, color=color)
    ax.axis('off')

plt.suptitle("黑盒迁移攻击实验结果", fontsize=15, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

# 详细分析
print("\n📈 详细迁移分析:")
print("-" * 75)
for name, result in transfer_results.items():
    conf_change = result['orig_conf'] - result['adv_conf']
    print(f"{name}:")
    print(f"  原始: 类别 {result['orig_pred']} ({result['orig_conf']:.2%})")
    print(f"  攻击: 类别 {result['adv_pred']} ({result['adv_conf']:.2%})")
    print(f"  置信度变化: {conf_change:+.2%}")
    print()

### 🤔 思考一下

1. **迁移规律**：观察结果，哪些模型更容易被迁移攻击？为什么？
2. **架构影响**：ResNet-34 和 VGG-16 哪个更容易被攻击？这与模型架构有什么关系？
3. **实际威胁**：在现实世界中，攻击者如何利用迁移性攻击黑盒系统？

## 第四部分：提高迁移性的策略

如何提高对抗样本的迁移成功率？我们将探索两种常用策略。

In [ ]:
### 策略 1：增大扰动强度

# 测试不同扰动大小对迁移性的影响
epsilon_values = [0.01, 0.03, 0.05, 0.1]

print("=" * 80)
print("📊 扰动强度对迁移性的影响")
print("=" * 80)
print(f"{'ε 值':<10} {'迁移成功率':<15} {'成功模型数':<15} {'平均置信度下降':<20}")
print("-" * 80)

epsilon_results = []
for eps in epsilon_values:
    # ========== 填空 3：使用不同的 epsilon 生成对抗样本 ==========
    # 
    # 🎯 任务：测试不同扰动强度的迁移效果
    # 
    # 💡 提示：
    #   - 使用 pgd_attack 函数
    #   - 参数中的 epsilon 改为当前循环变量 eps
    #   - 其他参数保持不变
    # 
    # 难度：⭐⭐☆☆☆
    
    adv = ___________  # 期望：pgd_attack(surrogate_model, original_image, surrogate_class, eps, 0.01, 20)
    
    # 测试迁移效果
    success_count = 0
    conf_changes = []
    
    for name, model in target_models.items():
        orig_pred, orig_conf = predict(model, original_image)
        adv_pred, adv_conf = predict(model, adv)
        
        if orig_pred != adv_pred:
            success_count += 1
        conf_changes.append(orig_conf - adv_conf)
    
    transfer_rate = success_count / len(target_models) * 100
    avg_conf_change = np.mean(conf_changes)
    
    epsilon_results.append({
        'eps': eps,
        'rate': transfer_rate,
        'count': success_count,
        'conf_change': avg_conf_change
    })
    
    print(f"{eps:<10.2f} {transfer_rate:<15.1f}% {f'{success_count}/{len(target_models)}':<15} {avg_conf_change:<20.2%}")

print("=" * 80)

# 可视化扰动强度影响
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

eps_list = [r['eps'] for r in epsilon_results]
rate_list = [r['rate'] for r in epsilon_results]
conf_list = [r['conf_change'] for r in epsilon_results]

ax1.plot(eps_list, rate_list, 'bo-', linewidth=2, markersize=8)
ax1.set_xlabel('扰动强度 ε', fontsize=12)
ax1.set_ylabel('迁移成功率 (%)', fontsize=12)
ax1.set_title('扰动强度 vs 迁移成功率', fontsize=13, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0, 110)

ax2.plot(eps_list, conf_list, 'ro-', linewidth=2, markersize=8)
ax2.set_xlabel('扰动强度 ε', fontsize=12)
ax2.set_ylabel('平均置信度下降', fontsize=12)
ax2.set_title('扰动强度 vs 置信度下降', fontsize=13, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
### 策略 2：多模型集成攻击（高级）

def ensemble_attack(models, image, labels, epsilon, alpha, num_steps):
    """
    多模型集成攻击：同时考虑多个模型的梯度
    
    核心思想：生成对多个模型都有效的对抗样本，提高迁移性
    
    参数:
        models: 模型列表
        image: 原始图像
        labels: 各模型的原始预测标签列表
        epsilon, alpha, num_steps: PGD 参数
    
    返回:
        adversarial_image: 对抗样本
    """
    adv_image = image.clone().detach()
    
    for step in range(num_steps):
        adv_image.requires_grad = True
        total_loss = 0
        
        # ========== 填空 4：累加多个模型的损失 ==========
        # 
        # 🎯 任务：计算多个模型的平均损失用于梯度计算
        # 
        # 💡 提示：
        #   - 遍历 models 和 labels（使用 zip）
        #   - 对每个模型计算损失并累加到 total_loss
        #   - 损失函数：nn.CrossEntropyLoss()
        # 
        # 难度：⭐⭐⭐☆☆
        
        for model, label in zip(models, labels):
            normalized = normalize(adv_image).unsqueeze(0)
            output = model(normalized)
            loss = ___________  # 期望：nn.CrossEntropyLoss()(output, torch.tensor([label]))
            total_loss += loss
        
        # 计算平均损失
        avg_loss = total_loss / len(models)
        
        # 反向传播
        for model in models:
            model.zero_grad()
        avg_loss.backward()
        
        gradient = adv_image.grad.data
        adv_image = adv_image.detach() + alpha * torch.sign(gradient)
        
        # 投影和裁剪
        perturbation = adv_image - image
        perturbation = torch.clamp(perturbation, -epsilon, epsilon)
        adv_image = torch.clamp(image + perturbation, 0, 1)
    
    return adv_image.detach()

# ========== 填空 5：构建集成攻击的模型列表 ==========
# 
# 🎯 任务：选择多个模型用于集成攻击
# 
# 💡 提示：
#   - 使用替代模型 + 一个目标模型（如 ResNet-34）
#   - 构建模型列表：[surrogate_model, target_models['ResNet-34']]
#   - 同时构建对应的标签列表
# 
# 难度：⭐⭐☆☆☆

ensemble_models = ___________  # 期望：[surrogate_model, target_models['ResNet-34']]

# 获取各模型的原始预测
ensemble_labels = [predict(m, original_image)[0] for m in ensemble_models]

# 执行集成攻击
print("\n" + "=" * 70)
print("🔬 多模型集成攻击")
print("=" * 70)
print(f"使用模型: ResNet-18 + ResNet-34")
print("正在生成集成对抗样本...")

ensemble_adv = ensemble_attack(ensemble_models, original_image, ensemble_labels, 
                                epsilon=0.03, alpha=0.01, num_steps=20)

# 对比单模型攻击和集成攻击的迁移效果
print("\n集成攻击 vs 单模型攻击迁移效果对比:")
print("-" * 70)
print(f"{'目标模型':<18} {'单模型攻击':<18} {'集成攻击':<18}")
print("-" * 70)

single_success = 0
ensemble_success = 0

for name, model in target_models.items():
    orig_pred, _ = predict(model, original_image)
    
    # 单模型攻击结果
    single_adv_pred, _ = predict(model, adversarial_image)
    single_status = "✓ 成功" if orig_pred != single_adv_pred else "✗ 失败"
    if orig_pred != single_adv_pred:
        single_success += 1
    
    # 集成攻击结果
    ensemble_adv_pred, _ = predict(model, ensemble_adv)
    ensemble_status = "✓ 成功" if orig_pred != ensemble_adv_pred else "✗ 失败"
    if orig_pred != ensemble_adv_pred:
        ensemble_success += 1
    
    print(f"{name:<18} {single_status:<18} {ensemble_status:<18}")

print("-" * 70)
print(f"迁移成功率对比:")
print(f"  单模型攻击: {single_success}/{len(target_models)} ({single_success/len(target_models)*100:.1f}%)")
print(f"  集成攻击:   {ensemble_success}/{len(target_models)} ({ensemble_success/len(target_models)*100:.1f}%)")
print("=" * 70)

## 📋 实验小结

### 核心收获

通过本实验，你已经：

1. **理解迁移性**：掌握了对抗样本跨模型迁移的原理和实践方法
2. **实现黑盒攻击**：学会了在不知道目标模型的情况下发起攻击
3. **分析影响因素**：探索了扰动强度对迁移成功率的影响
4. **掌握高级技巧**：实现了多模型集成攻击来提高迁移性

### 关键概念回顾

**迁移攻击流程**：
```
1. 准备阶段：获取/训练替代模型
2. 攻击阶段：在替代模型上生成对抗样本（白盒）
3. 迁移阶段：将对抗样本应用到目标模型（黑盒）
4. 评估阶段：测试迁移成功率
```

**提高迁移性的策略**：

| 策略 | 原理 | 效果 | 代价 |
|------|------|------|------|
| **增大扰动** | 更大的扰动覆盖更广的决策空间 | 提高成功率 | 扰动更明显 |
| **多模型集成** | 找到多个模型的共同脆弱点 | 显著提高迁移性 | 需要多个模型 |
| **迭代优化** | PGD 比 FGSM 更精确 | 中等提升 | 计算成本高 |
| **输入变换** | 数据增强（平移、缩放等） | 提高鲁棒性 | 实现复杂 |

**影响迁移性的因素**：
1. **模型架构相似度**：相似架构迁移率更高
2. **训练数据**：相同数据集训练的模型更易迁移
3. **扰动强度**：ε 越大迁移率越高，但越容易被发现
4. **攻击方法**：迭代方法（PGD）优于单步方法（FGSM）

### 实验观察

根据实验结果，通常可观察到：
- **架构相似性**：ResNet-18 → ResNet-34 的迁移率最高
- **扰动阈值**：存在一个 ε 值（通常 0.03-0.05），超过后迁移率显著提升
- **集成优势**：多模型集成攻击的迁移率比单模型提高 20-40%

### 实际应用场景

迁移攻击在现实世界的威胁：
- **API 攻击**：攻击者无法访问模型参数，只能通过 API 查询
- **商业系统**：目标模型是专有的黑盒系统
- **防御评估**：测试模型对未知攻击的鲁棒性

### 填空参考答案

<details>
<summary>点击查看所有填空参考答案</summary>

**填空 1：在替代模型上生成对抗样本**
```python
adversarial_image = pgd_attack(surrogate_model, original_image, surrogate_class, epsilon, alpha, num_steps)
```

**填空 2：获取对抗样本在目标模型上的预测**
```python
adv_pred, adv_conf = predict(model, adversarial_image)
```

**填空 3：使用不同的 epsilon 生成对抗样本**
```python
adv = pgd_attack(surrogate_model, original_image, surrogate_class, eps, 0.01, 20)
```

**填空 4：累加多个模型的损失**
```python
loss = nn.CrossEntropyLoss()(output, torch.tensor([label]))
```

**填空 5：构建集成攻击的模型列表**
```python
ensemble_models = [surrogate_model, target_models['ResNet-34']]
```

</details>

### 延伸学习

- **下一个实验**：实验 3.4 - 文本对抗攻击
- **论文阅读**：
  - Papernot et al., "Practical Black-Box Attacks against Machine Learning" (ACM Asia CCS 2017)
  - Liu et al., "Delving into Transferable Adversarial Examples" (ICLR 2017)
- **进阶挑战**：
  - 实现基于查询的黑盒攻击（无需替代模型）
  - 测试防御方法（对抗训练）对迁移攻击的效果
  - 探索物理世界的迁移攻击（打印的对抗样本）

### 🤔 深入思考

1. **防御策略**：如何防御迁移攻击？集成多个不同架构的模型会有帮助吗？
2. **检测方法**：能否检测出对抗样本是从其他模型迁移过来的？
3. **物理攻击**：迁移性对物理世界攻击（如对抗补丁）有什么启示？
4. **伦理问题**：迁移攻击的研究应该公开到什么程度？

### 安全提醒

⚠️ **本实验仅用于教育目的**

迁移攻击是评估黑盒系统安全性的重要方法。请将所学知识用于：
- 评估自己系统的鲁棒性
- 开发针对迁移攻击的防御方法
- 推进对抗机器学习安全研究

请勿用于：
- 攻击他人的商业 API 或服务
- 绕过安全系统
- 其他未经授权的活动